In [196]:
#importing necessary packages
from bs4 import BeautifulSoup
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

#Web Scrappping with Beautiful Soup
page_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page=requests.get(page_link)
soup = BeautifulSoup(page.content, 'html.parser')
table_soup = soup.find_all('table')[0] 
table = pd.read_html(str(table_soup))

#Creating the table dataframe 
data_frame=pd.DataFrame(table[0])
data_frame.columns=['Postcode','Borough','Neighborhood']
data_frame=data_frame.drop(0).reset_index(drop=True)

#### Removing the rows whose Borough Value is Not Assigned

In [197]:
#Deleting the Not Assigned in the Borough Rows
data_frame['Borough']=data_frame['Borough'].replace(to_replace='Not assigned' , value=np.nan)
data_frame = data_frame[pd.notnull(data_frame['Borough'])].reset_index(drop=True)

#### Combining the rows who has same postal code value

In [198]:
#combining the rows of the same postacode value
postal={}
for index,row in data_frame.iterrows():
    row=dict(row)
    if row['Postcode'] in postal:
        
        postal[row['Postcode']]['Neighborhood']=postal[row['Postcode']]['Neighborhood']+', '+row['Neighborhood']
        
    else:
        if row['Neighborhood']=='Not assigned':
            row['Neighborhood']=row['Borough']
        postal[row['Postcode']]=row 
df1=pd.DataFrame(postal).transpose()
df1.set_index('Postcode',inplace=True)
df1=df1.reset_index(drop=False)
df1.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [225]:
df1.shape[0]

103

In [200]:
df = pd.read_csv('http://cocl.us/Geospatial_data')
df.columns = ['PostalCode', 'Latitude', 'Longitude']   
df.sort_values(by='PostalCode',ascending=True,inplace=True)
df1.sort_values(by='Postcode',ascending=True,inplace=True)
df1['Latitude']=df['Latitude']
df1['Longitude']=df['Longitude']
df1.sort_index()
df1.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [201]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df1['Borough'].unique()),
        df1.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [202]:
toronto_data=df1[df1['Borough'].str.contains("East Toronto")].reset_index(drop=True)

In [203]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'East Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Totonto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Totonto are 43.653963, -79.387207.


In [78]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [204]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [205]:
#visualisation of Toronto
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

map_toronto=folium.Map(location=[latitude,longitude],zoom_start=11)
for lat,lng,label in zip(toronto_data['Latitude'],toronto_data['Longitude'],toronto_data['Neighborhood']):
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [206]:
CLIENT_ID = '33MAOBUFOQV4JD2DRIVJP1DKT5WKB4DOYVRUFQPJDEL3BMPU' # your Foursquare ID
CLIENT_SECRET = 'BWJAUUER4DPCGCTA2GOFY1IB1RAEFCNWHDDFFW0QNUGQ5XGH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 33MAOBUFOQV4JD2DRIVJP1DKT5WKB4DOYVRUFQPJDEL3BMPU
CLIENT_SECRET:BWJAUUER4DPCGCTA2GOFY1IB1RAEFCNWHDDFFW0QNUGQ5XGH


In [207]:
toronto_data.loc[0,'Neighborhood']
neighborhood_latitude = toronto_data.loc[0, 'Latitude']
neighborhood_longitude = toronto_data.loc[0, 'Longitude']

In [208]:
# type your answer here
radius=500
LIMIT=100
url='https://api.foursquare.com/v2/venues/explore?&client_id={0}&client_secret={1}&v={2}&ll={3},{4}&radius={5}&limit={6}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c2e2986db04f53ab9eff69e'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b8daea1f964a520480833e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d11b941735',
         'name': 'Pub',
         'pluralName': 'Pubs',
         'primary': True,
         'shortName': 'Pub'}],
       'id': '4b8daea1f964a520480833e3',
       'location': {'address': '676 Kingston Rd.',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Main St.',
        'distance': 460,
        'formattedAddress': ['676 Kingston Rd. (at Main St.)',
         'Toronto ON M4E 1R4',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
 

In [209]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4b8daea1f964a520480833e3-0,"[{'primary': True, 'shortName': 'Pub', 'name':...",4b8daea1f964a520480833e3,676 Kingston Rd.,CA,Toronto,Canada,at Main St.,460,"[676 Kingston Rd. (at Main St.), Toronto ON M4...","[{'lat': 43.679181434941015, 'label': 'display...",43.679181,-79.297215,M4E 1R4,ON,Grover Pub and Grub,0,[]
1,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4e77e3861f6ecf8d3648300c-1,"[{'primary': True, 'shortName': 'Coffee Shop',...",4e77e3861f6ecf8d3648300c,637 Kingston Rd.,CA,Toronto,Canada,at Main St.,486,"[637 Kingston Rd. (at Main St.), Toronto ON M4...","[{'lat': 43.67879837444001, 'label': 'display'...",43.678798,-79.298045,M4E 1R3,ON,Starbucks,0,[]
2,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-5c125491ad1789002d4eda71-2,"[{'primary': True, 'shortName': 'Astrologer', ...",5c125491ad1789002d4eda71,"Above RIDA Fashions, Behind PIZZA PIZZA (ALBIO...",CA,Toronto,Canada,NaN,429,"[Above RIDA Fashions, Behind PIZZA PIZZA (ALBI...","[{'lat': 43.680186658703796, 'label': 'display...",43.680187,-79.292337,M9V 1A6,ON,Guru Raghavendra Ji,0,[]
3,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4df91c4bae60f95f82229ad5-3,"[{'primary': True, 'shortName': 'Neighborhood'...",4df91c4bae60f95f82229ad5,NaN,CA,Toronto,Canada,NaN,468,"[Toronto ON, Canada]","[{'lat': 43.68056321147582, 'label': 'display'...",43.680563,-79.292869,NaN,ON,Upper Beaches,0,[]


In [210]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [211]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type,axis=1)
nearby_venues.columns=[col.split('.')[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Grover Pub and Grub,Pub,43.679181,-79.297215
1,Starbucks,Coffee Shop,43.678798,-79.298045
2,Guru Raghavendra Ji,Astrologer,43.680187,-79.292337
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [212]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={0}&client_secret={1}&v={2}&ll={3},{4}&radius={5}&limit={6}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [214]:
toronto_venues=getNearbyVenues(toronto_data['Neighborhood'],toronto_data['Latitude'],toronto_data['Longitude'])
toronto_venues


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,The Beaches,43.676357,-79.293031,Guru Raghavendra Ji,43.680187,-79.292337,Astrologer
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
5,"The Danforth West, Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
6,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
7,"The Danforth West, Riverdale",43.679557,-79.352188,Messini Authentic Gyros,43.677827,-79.350569,Greek Restaurant
8,"The Danforth West, Riverdale",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
9,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant


In [215]:
toronto_onehot=pd.get_dummies(toronto_venues['Venue Category'],prefix="",prefix_sep="")


In [216]:
toronto_onehot['Neighborhood']=toronto_venues['Neighborhood']
toronto_onehot = toronto_onehot.reindex(columns=(['Neighborhood'] + list([a for a in toronto_onehot.columns if a != 'Neighborhood']) ))
toronto_onehot.head()

,Neighborhood,American Restaurant,Astrologer,Auto Workshop,Bakery,Bank,Bar,Board Shop,Bookstore,Brewery,...,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Trail,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [217]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,American Restaurant,Astrologer,Auto Workshop,Bakery,Bank,Bar,Board Shop,Bookstore,Brewery,...,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Trail,Yoga Studio
0,Business reply mail Processing Centre969 Eastern,0.000000,0.00,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,...,0.000000,0.000000,0.055556,0.055556,0.055556,0.000000,0.000000,0.000000,0.00000,0.055556
1,Studio District,0.051282,0.00,0.000000,0.051282,0.025641,0.025641,0.000000,0.025641,0.025641,...,0.025641,0.025641,0.000000,0.000000,0.000000,0.025641,0.000000,0.000000,0.00000,0.025641
2,The Beaches,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
3,"The Beaches West, India Bazaar",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.047619,...,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.00000,0.000000
4,"The Danforth West, Riverdale",0.023810,0.00,0.000000,0.023810,0.000000,0.000000,0.000000,0.047619,0.023810,...,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.02381,0.023810


In [218]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [219]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Pizza Place,Auto Workshop,Brewery,Burrito Place,Butcher,Comic Shop,Farmers Market,Fast Food Restaurant
1,Studio District,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Bar,Convenience Store,Ice Cream Shop,Gym / Fitness Center,Gastropub
2,The Beaches,Astrologer,Coffee Shop,Pub,Yoga Studio,Coworking Space,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Dessert Shop
3,"The Beaches West, India Bazaar",Park,Pizza Place,Burger Joint,Pub,Ice Cream Shop,Italian Restaurant,Burrito Place,Coffee Shop,Liquor Store,Movie Theater
4,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Fruit & Vegetable Store,Indian Restaurant,Health Food Store,Grocery Store,Trail


In [220]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Pizza Place,Auto Workshop,Brewery,Burrito Place,Butcher,Comic Shop,Farmers Market,Fast Food Restaurant
1,Studio District,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Bar,Convenience Store,Ice Cream Shop,Gym / Fitness Center,Gastropub
2,The Beaches,Astrologer,Coffee Shop,Pub,Yoga Studio,Coworking Space,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Dessert Shop
3,"The Beaches West, India Bazaar",Park,Pizza Place,Burger Joint,Pub,Ice Cream Shop,Italian Restaurant,Burrito Place,Coffee Shop,Liquor Store,Movie Theater
4,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Fruit & Vegetable Store,Indian Restaurant,Health Food Store,Grocery Store,Trail


In [221]:
from sklearn.cluster import KMeans

kclusters = 3
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 


array([2, 2, 1, 2, 0], dtype=int32)

In [222]:
toronto_grouped.drop('Neighborhood', 1)

,American Restaurant,Astrologer,Auto Workshop,Bakery,Bank,Bar,Board Shop,Bookstore,Brewery,Bubble Tea Shop,...,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Trail,Yoga Studio
0,0.000000,0.00,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.00000,...,0.000000,0.000000,0.055556,0.055556,0.055556,0.000000,0.000000,0.000000,0.00000,0.055556
1,0.051282,0.00,0.000000,0.051282,0.025641,0.025641,0.000000,0.025641,0.025641,0.00000,...,0.025641,0.025641,0.000000,0.000000,0.000000,0.025641,0.000000,0.000000,0.00000,0.025641
2,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
3,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.047619,0.00000,...,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.00000,0.000000
4,0.023810,0.00,0.000000,0.023810,0.000000,0.000000,0.000000,0.047619,0.023810,0.02381,...,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.02381,0.023810


In [223]:
toronto_merged=toronto_data
toronto_merged['Cluster Labels'] = kmeans.labels_
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Astrologer,Coffee Shop,Pub,Yoga Studio,Coworking Space,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Dessert Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Fruit & Vegetable Store,Indian Restaurant,Health Food Store,Grocery Store,Trail
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Park,Pizza Place,Burger Joint,Pub,Ice Cream Shop,Italian Restaurant,Burrito Place,Coffee Shop,Liquor Store,Movie Theater
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Bar,Convenience Store,Ice Cream Shop,Gym / Fitness Center,Gastropub
4,M7Y,East Toronto,Business reply mail Processing Centre969 Eastern,43.662744,-79.321558,0,Light Rail Station,Yoga Studio,Pizza Place,Auto Workshop,Brewery,Burrito Place,Butcher,Comic Shop,Farmers Market,Fast Food Restaurant


In [224]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters